## 1. Man-in-the-middle Attack
---

SSH之所以能够保证安全，原因在于它采用了公钥加密。

整个过程是这样的：（1）Remote Host收到用户的登录请求，把自己的公钥发给用户。（2）用户使用这个公钥，将登录密码加密后，发送回来。（3）远程主机用自己的私钥，解密登录密码，如果密码正确，就同意用户登录。

这个过程本身是安全的，但是实施的时候存在一个风险：如果有人截获了登录请求，然后冒充远程主机，将伪造的公钥发给用户，那么用户很难辨别真伪。因为不像https协议，SSH协议的公钥是没有证书中心（CA）公证的，也就是说，都是自己签发的。

可以设想，如果攻击者插在用户与远程主机之间（比如在公共的wifi区域），用伪造的公钥，获取用户的登录密码。再用这个密码登录远程主机，那么SSH的安全机制就荡然无存了。这种风险就是著名的"中间人攻击"（Man-in-the-middle attack）。

SSH协议是如何应对的呢？看来面的例子就知道了, 

在Mac上通过ssh连接远程的服务器, 第一次连接的时候会问下面提示:

```shell
ssh root@144.202.16.29        
The authenticity of host '144.202.16.29 (144.202.16.29)' can't be established.
ED25519 key fingerprint is SHA256:sa5vDYS0yhdMRXO6CgMrp9AcQoVQRiDw6TnzTKesnzQ.
This key is not known by any other names
Are you sure you want to continue connecting (yes/no/[fingerprint])? yes
....
```

首先通过上面信息我们可以得到三个信息: 

- 该remote host上的shh使用的ED25519算法生成的public/private key, 也就是说ED25519是个public-key cryptography 算法, 常见的public-key cryptography算法还有RSA, 所以RSA和ED25519是并列的. 
    - Today, the RSA is the most widely used public-key algorithm for SSH key. But compared to Ed25519, it's slower and even considered not safe if it's generated with the key smaller than 2048-bit length.
- 该key fingerprinter是由SHA256 hash function生成的, 另外常见的hash function还有md5
- key fingerprinter的值是sa5vDYS0yhdMRXO6CgMrp9AcQoVQRiDw6TnzTKesnzQ (key fingerprint其实就是remote host公钥指纹, 可以理解为公钥的缩短版, 以便于我们比对)

> In public-key cryptography, a public key fingerprint is a short sequence of bytes used to identify a longer public key. Fingerprints are created by applying a cryptographic hash function to a public key. Since fingerprints are shorter than the keys they refer to, they can be used to simplify certain key management tasks. 

上面的提醒是在说无法确认host主机的真实性，只知道它的key fingerprint，问你还想继续连接吗(注意那个establish还有个认证确认的意思, 并不知识建立的意思)?

很自然的一个问题就是，用户怎么知道远程主机(remote host)的公钥指纹应该是多少？回答是没有好办法, 你可以在你购买VPS的网站上连接自己服务器, 然后输入下面命令, 查看自己服务器上的key fingerprint:

```zsh
[root@vultr ssh]# ssh-keygen -lf /etc/ssh/ssh_host_ed25519_key.pub
256 SHA256:sa5vDYS0yhdMRXO6CgMrp9AcQoVQRiDw6TnzTKesnzQ root@vultr.guest (ED25519)
```

如果输出内容与上面登录时给的key fingerprint相同, 所以我们要来接的这个是我们的真正主机, 所以并不是中间人攻击, hhh

注意因为上面提示的是`ED25519 key fingerprint is...`, 所以我在server查看key fingerprint输入的是查看`ssh_host_ed25519_key.pub`文件, 如果你的remote host使用的是其他hash function生成的key fingerprint, 那你就要查看其他文件了:

```shell
ssh-keygen -E md5 -lf /etc/ssh/ssh_host_ed25519_key.pub
...
```

了解更多:

- [How to check your SSH key fingerprint (verify the authenticity of the remote host)](https://bitlaunch.io/blog/how-to-check-your-ssh-key-fingerprint/)

## 2. 验证公私钥位置
-----

这个时候我们在电脑终端输入yes, 然后就会提示输入密码(你要连接的remote host上的某个用户, 比如root的对应密码), 然后系统会提示如下:

```shell
Warning: Permanently added '144.202.16.29' (ED25519) to the list of known hosts.
root@144.202.16.29's password: 
Last login: Fri Apr  7 20:36:06 2023
[root@vultr ~]# ls
```

当远程主机的公钥被接受以后，它就会被保存在文件`~/.ssh/known_hosts`之中。下次再连接这台主机，系统就会认出它的公钥已经保存在本地了，从而跳过警告部分，直接提示输入密码。

我们来查看Mac上的输出, 

```zsh
ls ~/.ssh 
id_rsa          id_rsa.pub      known_hosts     known_hosts.old

cat ~/.ssh/known_hosts 
github.com ssh-ed25519 AAAAC3NzaC1lZDI1N...
github.com ecdsa-sha2-nistp256 AAAAE2VjZH....
github.com ssh-rsa AAAAB3NzaC1yc2EAAAADA....

144.202.16.29 ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIOpG35RMxOKeeLbTfdWlPgToThzrm00sRpMRQs+pdYig
144.202.16.29 ecdsa-sha2-nistp256 AAAAE2VjZHNhLXNoYTItbmlzdHAyNTYAAAAIbmlzdHAyNTYAAABBBHP5xEl1122X1Vtc5LzqMp6vlvd4cHRD151ag61xXThvT7KM9vuUK23ol4LKoXMoivUH1SAcWandumVKG37zZfA=
```

再看看服务器上的输出, 

```
[root@vultr ssh]# ls /etc/ssh
moduli            ssh_host_dsa_key.pub    ssh_host_ed25519_key.pub
ssh_config        ssh_host_ecdsa_key      ssh_host_rsa_key
sshd_config       ssh_host_ecdsa_key.pub  ssh_host_rsa_key.pub
ssh_host_dsa_key  ssh_host_ed25519_key

[root@vultr ssh]# cat ssh_host_ed25519_key.pub 
ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIOpG35RMxOKeeLbTfdWlPgToThzrm00sRpMRQs+pdYig root@vultr.guest

[root@vultr ssh]# cat ssh_host_ecdsa_key.pub 
ecdsa-sha2-nistp256 AAAAE2VjZHNhLXNoYTItbmlzdHAyNTYAAAAIbmlzdHAyNTYAAABBBHP5xEl1122X1Vtc5LzqMp6vlvd4cHRD151ag61xXThvT7KM9vuUK23ol4LKoXMoivUH1SAcWandumVKG37zZfA= root@vultr.guest
```

这两个一个是公钥, 一个是公钥的指纹, 可以和上面Mac的存储的内容是一样. 

然后服务器这边, 你看上面的输出, 有`ssh_host_rsa_key.pub`, `ssh_host_ecdsa_key.pub`这就是不同算法产生的不同的key, 上面介绍的rsa和ED25519算法~

------

## 3. 公钥登录
---

使用密码登录，每次都必须输入密码，非常麻烦。好在SSH还提供了公钥登录，可以省去输入密码的步骤。

所谓"公钥登录"，原理很简单，就是用户将自己的公钥储存在远程主机上。登录的时候，远程主机会向用户发送一段随机字符串，用户用自己的私钥加密后，再发回来。远程主机用事先储存的公钥进行解密，如果成功，就证明用户是可信的，直接允许登录shell，不再要求密码。

这种方法要求用户必须提供自己的公钥。如果没有现成的，可以直接用ssh-keygen生成一个：

```zsh
ssh-keygen
```

运行上面的命令以后，系统会出现一系列提示，可以一路回车。其中有一个问题是，要不要对私钥设置口令（passphrase），如果担心私钥的安全，这里可以设置一个。

运行结束以后，在`~/.ssh/`目录下，会新生成两个文件：`id_rsa.pub`和`id_rsa`, 前者是你的公钥，后者是你的私钥。

这时再输入下面的命令，将公钥传送到远程主机host上面：

```zsh
$ ssh-copy-id root@144.202.16.29
/usr/bin/ssh-copy-id: INFO: Source of key(s) to be installed: "/Users/shaowen/.ssh/id_rsa.pub"
/usr/bin/ssh-copy-id: INFO: attempting to log in with the new key(s), to filter out any that are already installed
/usr/bin/ssh-copy-id: INFO: 1 key(s) remain to be installed -- if you are prompted now it is to install the new keys
root@144.202.16.29's password: 

Number of key(s) added:        1

Now try logging into the machine, with:   "ssh 'root@144.202.16.29'"
and check to make sure that only the key(s) you wanted were added.
```

好了，从此你再登录，就不需要输入密码了, 直接输入, 即可登录

```
$ ssh root@144.202.16.29
```

## 4. authorized_keys文件
----

Remote Host将用户的公钥，保存在登录后的用户主目录的`~/.ssh/authorized_keys`中。公钥就是一段字符串，只要把它追加在`authorized_keys`的末尾就行了。

```zsh
[root@vultr ~]# ls -a
.   .bash_history  .bash_profile  .cache  .pki  .tcshrc
..  .bash_logout   .bashrc        .cshrc  .ssh

[root@vultr ~]# cat .ssh/authorized_keys 
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDvbHLCIxxUDDqktbqdrICPa+JDd3kEyowKpy9igugi7R+f/94UDBDJLmeu+K8wi90pjwq+mTM6bSPXBkjmYGibCPbUKk7RtrVx5FdR488PR7/ptMqQXJeQeMOIXvK2Lfnzay+rH5Fg/8z1+pd7cuHPq0bWm5LroGq+bYXVTIYgjKC5NDxPbQCY7zd4c0L+SvxlwqrJFvRBZKY41UBLywtuM8geluLWaGcbikX1K2hFVcZ7ETogG7eqdRBtbfx+JxhyRY1Od+snM88CSfuQkOgs4xQli3GrGttgY0f8BA65/pbixG9gAPkacEkexS997iuTP9BmwLmwWq1pw91c0yEQO1JnsbGHj/YfRhBV6s4FL8n5uVC0My64tisqA+8eZTeld8Zwem4XQGjoqwt2HYy1YXv0kOU8NyI0EGDz3fmqER3ex0cL+MqvWf/cnWQ6MRvGI3w/gL3+V8ueZv5qXpnY+ZH2UcrqEv7Xl74fkdqPYo53ySLQ9ZCiCitHgMjl3bk= shwezu@qq.com
```

在Mac上查看我自己的公钥, 是一样的:

```zsh
# cat id_rsa.pub 
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDvbHLCIxxUDDqktbqdrICPa+JDd3kEyowKpy9igugi7R+f/94UDBDJLmeu+K8wi90pjwq+mTM6bSPXBkjmYGibCPbUKk7RtrVx5FdR488PR7/ptMqQXJeQeMOIXvK2Lfnzay+rH5Fg/8z1+pd7cuHPq0bWm5LroGq+bYXVTIYgjKC5NDxPbQCY7zd4c0L+SvxlwqrJFvRBZKY41UBLywtuM8geluLWaGcbikX1K2hFVcZ7ETogG7eqdRBtbfx+JxhyRY1Od+snM88CSfuQkOgs4xQli3GrGttgY0f8BA65/pbixG9gAPkacEkexS997iuTP9BmwLmwWq1pw91c0yEQO1JnsbGHj/YfRhBV6s4FL8n5uVC0My64tisqA+8eZTeld8Zwem4XQGjoqwt2HYy1YXv0kOU8NyI0EGDz3fmqER3ex0cL+MqvWf/cnWQ6MRvGI3w/gL3+V8ueZv5qXpnY+ZH2UcrqEv7Xl74fkdqPYo53ySLQ9ZCiCitHgMjl3bk= shwezu@qq.com
```